In [ ]:
%tensorflow_version 1.x
!pip install segmentation-models==0.2.1

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing
from segmentation_models.losses import jaccard_loss
from segmentation_models.metrics import iou_score
from keras.callbacks import *
from sklearn.metrics import jaccard_score

In [ ]:
x = np.load('x_train_test.npy')
y = np.load('y_train_test.npy')
x_val=np.load('x_validation.npy')
y_val=np.load('y_validation.npy')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 103)
y_train = np.reshape(y_train,(1424,256,256,1))
y_test = np.reshape(y_test,(356,256,256,1))

In [ ]:
print('Training Images(x_train) Shape: ',x_train.shape)
print('Training Labels(y_train) Shape: ',y_train.shape)
print('Testing Images(x_test) Shape: ',x_test.shape)
print('Testing Labels(y_test) Shape: ',y_test.shape)
print('Validation Images(x_test) Shape: ',x_val.shape)
print('Validation Labels(y_test) Shape: ',y_val.shape)

In [ ]:
def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [ ]:
'''
Backbones used for model evaluations:
1. vgg19
2. seresnet18
3. resnet152
4. densenet121
5. mobilenet
'''
BACKBONE = 'REPLACE_BACKBONE_NAME_HERE'
preprocess_input = get_preprocessing(BACKBONE)
model = Unet(BACKBONE,encoder_weights=None)
model.compile(optimizer="Adam", loss= jaccard_distance_loss, metrics=['accuracy',iou_score])

In [ ]:
model.fit(
    x=x_train,
    y=y_train,
    batch_size=16,
    epochs=50,
    validation_data = (x_test,y_test)
)

In [ ]:
model.save("/content/drive/My Drive/seresnet18_coco.h5")

In [ ]:
res=model.predict(new_x_val)
print('IoU(Jaccard Score) :',round(jaccard_score(new_y_val.flatten(),res.flatten().round()),4))